In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import ipaddress

# --- 1. Convert IP addresses to float64 ---
def ip_to_float(ip):
    try:
        return float(int(ipaddress.IPv4Address(ip)))/1e9
    except:
        return 0.0  # for invalid or empty IPs
    
print(ip_to_float("192.17.31.2"))

0.3222347522


In [ ]:
unique_values = df["label"].unique().compute()

print(unique_values)

In [6]:
import pandas as pd

df = pd.DataFrame({
    'proto': ['tcp', 'udp', 'tcp']
})

df_encoded = pd.get_dummies(df, columns=['proto'], prefix=['proto'], dtype=int)
print(df_encoded)

   proto_tcp  proto_udp
0          1          0
1          0          1
2          1          0


> 0                           C&C-Mirai
0                    C&C-FileDownload
0          C&C-HeartBeat-FileDownload
0           PartOfAHorizontalPortScan
0                        Okiru-Attack
0                        FileDownload
0                                 C&C
0                           C&C-Torii
0                               Okiru
0                              Attack
0    PartOfAHorizontalPortScan-Attack
0                                DDoS
0       C&C-PartOfAHorizontalPortScan
0                C&C-HeartBeat-Attack
0                       C&C-HeartBeat
0                                   -
Name: detailed-label, dtype: string

In [ ]:
attack_types = [C&C-Mirai, C&C-FileDownload, C&C-HeartBeat-FileDownload, PartOfAHorizontalPortScan, Okiru-Attack, FileDownload, C&C, /
                C&C-Torii, Okiru, Attack, PartOfAHorizontalPortScan-Attack, DDoS, C&C-PartOfAHorizontalPortScan, C&C-HeartBeat-Attack, C&C-HeartBeat, "-"]

print(len(attack_types))

NameError: name 'C' is not defined

In [5]:
from datetime import datetime
import hashlib

start =datetime.now()
def string_to_float(s):
    return int(hashlib.sha256(s.encode('utf-8')).hexdigest(), 16) % 10**6 / 1e6

for index in range(10000):
    temp = string_to_float("ShA")
end = datetime.now()

time = str((end - start))
print(time)

0:00:00.014545


In [ ]:
pipeline
from sklearn.pipeline import Pipeline

# Pipeline: Standardize -> Normalize
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('normalizer', Normalizer(norm='l2'))
])

df_processed = pd.DataFrame(pipeline.fit_transform(df), columns=df.columns)

print("/nStandardized + L2 Normalized Data:")
print(df_processed)

In [ ]:
import dask.dataframe as dd
import numpy as np
from dask.dataframe import isna
# from dask.dataframe import notna
# Tạo DataFrame mẫu
df = dd.from_pandas(pd.DataFrame({
    'A': [1, np.nan, 3],
    'B': [np.nan, np.nan, np.nan],
    'C': ['x', 'y', np.nan]
}), npartitions=2)

# Lọc cột có dữ liệu
non_empty_cols = df.columns[(~isna(df)).any().compute()]
df = df[non_empty_cols]
print(df.shape[0].compute())

3


In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import hashlib
from dask_ml.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from dask.dataframe import isna
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
import os

# Tạo DataFrame Dask từ Pandas DataFrame
df = dd.from_pandas(pd.DataFrame({
    'col1': [1, 2, 3, 4],
    'col2': [0.2, 0.2121314, 0.1 ,0.6],
    'col3': [5,6,7,8],
    # 'col4': [np.nan, np.nan, np.nan, np.nan]
}), npartitions=1)


df = df.replace(["(empty)", "-"], np.nan)
df = df.drop_duplicates()

print("Before: /n",df.compute())


df = df.categorize(columns=['col1', 'col3'])
print(df.dtypes)
df= dd.get_dummies(
    df, 
    columns=['col1', 'col3'],
    prefix={'col1': 'proto', 'col3':'service'},  # Custom prefix
    # dummy_na=True,  # Tạo cột cho giá trị NaN
    # drop_first=True,  # Bỏ đi một cột để tránh đa cộng tuyến
    dtype=int
)


df = df.fillna("0")

numeric_cols = ['col2']

# scaler = StandardScaler()
# df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
scaler = StandardScaler()
def Standard_Scaling(partition):
    partition[numeric_cols] = scaler.fit_transform(partition[numeric_cols])
    return partition
df = df.map_partitions(Standard_Scaling, meta= dict(df.dtypes))
print("After Scaler: /n", df.compute())

# def normalize_partition(partition):
#     normalizer = Normalizer(norm='l2')
#     partition[numeric_cols] = normalizer.fit_transform(partition[numeric_cols])
#     print(partition)
#     return partition

# print(dict(df.dtypes))

# df = df.map_partitions(normalize_partition, meta=dict(df.dtypes))

    # def sum_of_squares(partition):
    #     return pd.Series([(partition ** 2).sum()])

    # Tính tổng bình phương theo từng partition, rồi cộng lại
    # sum_squares = df['col2'].map_partitions(sum_of_squares).sum().compute()
    # l2_norm = np.sqrt(sum_squares)
    # df['col2'] = df['col2'] / l2_norm

def normalize_partition(pdf):
    row_norm = np.sqrt((pdf**2).sum(axis=1))
    row_norm[row_norm == 0] = np.nan
    return pdf.div(row_norm, axis=0)
cols = ['col2', 'proto_1']
df[cols] = df[cols].map_partitions(normalize_partition, meta=dict(df[cols].dtypes))

def col_is_all_nan(df, col):
    return df[col].isna().all()

# print(col_is_all_nan(df, 'col2'))
# # Đánh giá từng cột, từng partition
# results = []
# for col in df.columns:
#     # Tính isna().all() trên từng partition
#     parts = df.map_partitions(col_is_all_nan, col=col, meta={col: 'bool'}).compute()
    
#     # Nếu tất cả partition trả về True => cột này toàn NaN
#     if parts.all():
#         results.append(col)

# # Drop các cột rỗng toàn phần
# df = df.drop(columns=results)

print("After normal/n", df.compute())

output_file ="temp.csv"


header_saved= False
for i, partition in enumerate(df.partitions):
    print("Solving ", i)
    partition.compute().to_csv(output_file, index=False, mode="a", header= not  header_saved)  # Append không ghi header
    header_saved=True
# Tính toán và xem kết quả

# df = dd.read_csv('temp.csv')
# print(df)


Before: 
    col1      col2  col3
0     1  0.200000     5
1     2  0.212131     6
2     3  0.100000     7
3     4  0.600000     8
col1    category
col2     float64
col3    category
dtype: object
After Scaler: 
        col2  proto_1  proto_2  proto_3  proto_4  service_5  service_6  \
0 -0.408736        1        0        0        0          1          0   
1 -0.345192        0        1        0        0          0          1   
2 -0.932535        0        0        1        0          0          0   
3  1.686463        0        0        0        1          0          0   

   service_7  service_8  
0          0          0  
1          0          0  
2          1          0  
3          0          1  
After normal
        col2   proto_1  proto_2  proto_3  proto_4  service_5  service_6  \
0 -0.378351  0.925662        0        0        0          1          0   
1 -1.000000  0.000000        1        0        0          0          1   
2 -1.000000  0.000000        0        1        0         

In [2]:
s = "s"
print(type(str(s)))

<class 'str'>


In [59]:
import dask.dataframe as dd
import pandas as pd

# Tạo Dask DataFrame từ Pandas
pdf = pd.DataFrame({'x': range(10), 'y': range(10, 20)})
ddf = dd.from_pandas(pdf, npartitions=2)

# Hàm xử lý từng partition (hàm nhận và trả về Pandas DataFrame)
def custom_func(df):
    df['z'] = df['x'] + df['y']
    return df

# Áp dụng hàm qua từng partition
ddf_new = ddf.map_partitions(custom_func)

# Thực thi và in kết quả
print(ddf_new.compute())

   x   y   z
0  0  10  10
1  1  11  12
2  2  12  14
3  3  13  16
4  4  14  18
5  5  15  20
6  6  16  22
7  7  17  24
8  8  18  26
9  9  19  28


In [24]:
import pandas as pd
import dask.dataframe as dd
from sklearn.preprocessing import StandardScaler, Normalizer
import hashlib
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.pipeline import Pipeline
import ipaddress
import os
import numpy as np

# Khởi tạo scaler
scaler = StandardScaler()
normalizer = Normalizer()

# Cấu hình
chunk_size = 1000000
input_file = "/mnt/c/Users/hoang/FileCSV_DACN_2025/bigTemp.csv"
output_file = "C:/Users/hoang/FileCSV_DACN_2025/iot23_cleaned.csv"

cols_to_drop = ['uid', 'ts', 'tunnel_parents', 'local_orig', 'local_resp']
ip_to_float_Cols = ['id.orig_h', 'id.resp_h']
str_to_float_Cols = ['conn_state', 'history', 'id.resp_p', 'id.orig_p']
numeric_cols = ['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes']
onehot_cols = ['proto', 'service']

label_mapping = {"benign": 0, "Benign": 0, "Malicious": 1, "malicious": 1}

df = dd.read_csv(input_file)
df = df.replace([ "-", '-'], np.nan)
df = df.fillna(0)
df = df.astype({'duration': 'float32'})
print(df['duration'].unique().compute())


# ##### FUNC ############
# def ip_to_float(ip):
#     try:
#         return float(int(ipaddress.IPv4Address(ip)))/1e9
#     except:
#         return 0.0  # for invalid or empty IPs
    
# def sum_of_squares(partition):
#     return pd.Series([(partition ** 2).sum()])

# def string_to_float(s):
#     if pd.notna(s):
#         return int(hashlib.sha256(str(s).encode('utf-8')).hexdigest(), 16) % 10**8 / 1e8
#     return 0
# ##### FUNC ##############

# proto_categories =set()
# service_categories = set()

# for index, chunk in enumerate(pd.read_csv(input_file, chunksize=chunk_size, dtype='str')):
#     proto_categories.update(chunk['proto'].unique())
#     service_categories.update(chunk['service'].unique())
#     print(index)
    
# for index, chunk in enumerate(pd.read_csv(input_file, chunksize=chunk_size, dtype='str')):
#     chunk['proto'] = pd.Categorical(chunk['proto'], categories=proto_categories)
#     chunk['service'] = pd.Categorical(chunk['service'], categories=service_categories)
    
#     chunk = pd.get_dummies(chunk,columns=onehot_cols, prefix =onehot_cols, dtype='int32')
#     print(chunk.describe)
    
#     if index == 10:
#         break

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+----------+--------+----------+
| Column   | Found  | Expected |
+----------+--------+----------+
| duration | object | float64  |
+----------+--------+----------+

The following columns also raised exceptions on conversion:

- duration
  ValueError("could not convert string to float: '-'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'duration': 'object'}

to the call to `read_csv`/`read_table`.

In [ ]:
print(df['conn_state'].value_counts().compute())

conn_state
S2              267
OTH         3715156
SH              380
RSTRH          1341
S1              220
SHR              62
RSTR           2282
SF            52281
S0        319354204
RSTO           3712
S3             2485
REJ           51509
RSTOS0      2126047
Name: count, dtype: int64


In [7]:
print(df['orig_pkts'].value_counts().compute())

orig_pkts
154        9
24       342
46       445
234     1082
171       43
        ... 
5100       1
265      253
328        1
205      781
8295       1
Name: count, Length: 381, dtype: int64


In [18]:
print(df['duration'].value_counts().compute())

ValueError: could not convert string to float: '-'

: 

In [2]:
import dask.dataframe as dd
import numpy as np
import os
import pandas as pd

#### Thay đổi hiển thị ####
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')
pd.set_option('display.max_row', None)
pd.set_option('display.max_columns', None)
#### Change display ####

dir = ["C:/Users/hoang/FileCSV_DACN_2025/fl_IoT23" ,"/mnt/c/Users/hoang/FileCSV_DACN_2025/fl_IoT23"]
dir = dir[0] if os.name=='nt' else dir[1]
files = [dir +f"/parquet_f{index}" for index in range(3)]
print(files)

dfs = [dd.read_parquet(files[index]) for index in range(3)]

for df in dfs:
    print(f"Tỷ lệ file {df} ", df['label'].value_counts().compute())


['C:/Users/hoang/FileCSV_DACN_2025/fl_IoT23/parquet_f0', 'C:/Users/hoang/FileCSV_DACN_2025/fl_IoT23/parquet_f1', 'C:/Users/hoang/FileCSV_DACN_2025/fl_IoT23/parquet_f2']
Tỷ lệ file Dask DataFrame Structure:
              id.orig_h id.orig_p id.resp_h id.resp_p duration orig_bytes resp_bytes conn_state missed_bytes  history orig_pkts orig_ip_bytes resp_pkts resp_ip_bytes  label detailed-label proto_icmp proto_tcp proto_udp service_dhcp service_dns service_http service_irc service_ssh service_ssl
npartitions=8                                                                                                                                                                                                                                                                                       
                float64   float64   float64   float64  float64    float64    float64    float64      float64  float64   float64       float64   float64       float64  int64          int64      int64     int64